# The Anatomy of a Python Class

> Author: Samuel Farrens (<samuel.farrens@cea.fr>)  
> Year: 2019

Classes are one of the fundamental building blocks of Python and are essential for object-oriented programming.

In this notebook we will explore how classes work and look at tips and tricks for getting the most out of them. By the end of this tutorial you should have, not only a much better understanding of Python classes, but also some new ideas for writing better code.

## Contents

---

* [1 Disecting a Class](#1-Disecting-a-Class)
  * [The Class Dictionary](#The-Class-Dictionary)
  * [Instantiation](#Instantiation)
* [2 Methods](#2-Methods)
  * [Functions as Attributes](#Functions-as-Attributes)
  * [Static Methdos](#Static-Methods)
  * [Class Methods](#Class-Methods)
  * [Instance Methods](#Instance-Methods)
  * [Mixing Methods](#Mixing-Methods)
* [3 Properties](#3-Properties)
  * [Getters and Setters](#Getters-and-Setters)
  * [Class Properties](#Class-Properties)

## 1 Disecting a Class

---

To get a better grip on what a class is and how it works lets have a look at what is going on "inside". We can start by defining our first class and looking at its properties.

Classes are defined with the keyword `class`, much like the keyword `def` is used for defining functions. In the following cell we define a class without any attributes (note the use of the null `pass` statement).

In [1]:
# Define a dummy class
class myClass:
    pass

This object clearly has little use, but it is a good starting place to look at the structure of classes in general. Let's look at the `help` for this class.

In [2]:
help(myClass)

We can see that despite not setting attributes we have two default attributes predefined, namely `__dict__` and `__weakref__`. `__weakref__` is a special attribute that lists <a href="https://mindtrove.info/python-weak-references/" target="_blank">*weak references*</a> to the class object, a topic outside the scope of this notebook. Instead, we will focus on the `__dict__` attribute.

<br/>

### The Class Dictionary

Let's begin by printing the contents and type of `__dict__` for our dummy class.

In [3]:
# Print the class dictionary
print('myClass.__dict__ =', myClass.__dict__)
print('')
print('__dict__ is of type', type(myClass.__dict__))

This attribute is a *mappingproxy*, which is a special type of dictionary that does not have a `__setattr__` method (we will come back to this). In addition to the default attributes discussed above we can see:

* `'__doc__'` : The class docstring (defaults to `None`).
* `'__module__'` : The name of module in which the class was defined.

Now, let's see what happens when we assign a new attribute to the class. Class attributes are accessed and set using a dot (`.`) following the class name.

> Note that any Python object can be assigned as a class attribute. We will come back to this concept later!

In [4]:
# Assign the attribute myattr with value True to the class
myClass.myattr = True

# Print the class dictionary
print(myClass.__dict__)

We can see that the class dictionary has a new entry with the key `myattr` and corresponding value `True`. We can even demonstrate that this dictionary behaves as any other...

In [5]:
# Print the class dictionary with key 'myattr'
print(myClass.__dict__['myattr'])

...except that it does not allow assignment, given the absence of the `__setattr__` method.

In [6]:
# Assign a new value to 'myattr' in the class dictionary
myClass.__dict__['myattr'] = False

TypeError: 'mappingproxy' object does not support item assignment

Now let's define a new class with some predefined attributes and see what changes.

In [7]:
# Define a new dummy class with some attributes
class myClass:
    """ This is my class 
    """

    mybool = True
    myint = 1
    myfloat = 1.0
    mystring = 'string'
    
# Print the class dictionary    
print(myClass.__dict__)

{'__module__': '__main__', '__doc__': ' This is my class \n    ', 'mybool': True, 'myint': 1, 'myfloat': 1.0, 'mystring': 'string', '__dict__': <attribute '__dict__' of 'myClass' objects>, '__weakref__': <attribute '__weakref__' of 'myClass' objects>}


All of the attributes can be found in the class dictionary. Notice that `__doc__` is now a string. 

All of these attributes can be accessed...

In [8]:
# Print the class attributes
print('mybool =', myClass.mybool)
print('myint =', myClass.myint)
print('myfloat =', myClass.myfloat)
print('mystring =', myClass.mystring)
print('mydoc =', myClass.__doc__)

mybool = True
myint = 1
myfloat = 1.0
mystring = string
mydoc =  This is my class 
    


...and modified.

In [9]:
# Modify the value of the string attribute
myClass.mystring = 'my new string'

# Print the string attribute
print(myClass.mystring)

my new string


In summary the class `__dict__` attribute is useful for understanding how class attributes are stored, but should not really be used directly.

<br/>

### Instantiation

While accessing class attributes directly can be useful, in most applications it will be necessary to create a *class instance* (also reffered to simply as *objects*). Every time a class is isntantiated a unique Python class object is created. These objects retain the global class attributes but can also be assigned unique instance attributes.

We can create a class instance by calling the class name followed by `()`.

In [10]:
# Create an instance of the class
myinst = myClass()

# Print the instance dictionary
print('myinst.__dict__ =', myinst.__dict__)
print('')
print('__dict__ is of type', type(myinst.__dict__))

myinst.__dict__ = {}

__dict__ is of type <class 'dict'>


Here we see that the instance dictionary is a true dictionary and is empty, which simply means that no *instance attributes* have been set. 

However, instances still have access the class attributes.

In [11]:
# Print the instance class dictionary
print(myinst.__class__.__dict__)

{'__module__': '__main__', '__doc__': ' This is my class \n    ', 'mybool': True, 'myint': 1, 'myfloat': 1.0, 'mystring': 'my new string', '__dict__': <attribute '__dict__' of 'myClass' objects>, '__weakref__': <attribute '__weakref__' of 'myClass' objects>}


As with the class itself, attributes can be assigned to the instance object. 

In [12]:
# Assign an attribute to the instance
myinst.newbool = False

# Print the instance dictionary
print('myinst.__dict__ =', myinst.__dict__)
print('')
print('myClass.__dict__ =', myClass.__dict__)

myinst.__dict__ = {'newbool': False}

myClass.__dict__ = {'__module__': '__main__', '__doc__': ' This is my class \n    ', 'mybool': True, 'myint': 1, 'myfloat': 1.0, 'mystring': 'my new string', '__dict__': <attribute '__dict__' of 'myClass' objects>, '__weakref__': <attribute '__weakref__' of 'myClass' objects>}


We can see that the instance attribute has no impact on the class attributes.

In order to preassign instance attributes we will need to define a special initialisation method (`__init__`), which takes the special variable `self` as an argument.

In [14]:
# Define a new dummy class with an init method
class myClass:
    
    def __init__(self):
        
        self.myfloat = 3.5
   
# Create an instance of the class and print the attribute properties
myinst = myClass()
print('myinst.__dict__ =', myinst.__dict__)
print('myfloat =', myinst.myfloat)

myinst.__dict__ = {'myfloat': 3.5}
myfloat = 3.5


`self` represents any given class instance and attributes are assigned to it as usual with a dot. Printing the instance dictionary (which is equivalent to `self.__dict__`) reveals the attributes that have been assigned to it.

As with class attributes, instance attribute values can be modified.

In [15]:
myinst.myfloat = 7.2
print(myinst.__dict__)
print('myfloat =', myinst.myfloat)

{'myfloat': 7.2}
myfloat = 7.2


Additional agruments can be passed to `__init__` allowing for a more instance specific initialisation of the class.

In [16]:
# Define a new dummy class with a more dynamic init method
class myClass:
    
    def __init__(self, value):
        
        self.myfloat = value

# Create instances of the class and print the attribute properties
myinst1 = myClass(3.14)
myinst2 = myClass(6.28)
print('myfloat =', myinst1.myfloat)
print('myfloat =', myinst2.myfloat)

myfloat = 3.14
myfloat = 6.28


An important detail to take into account is that instance attributes will overload class attributes with the same name, but only for that instance object.

In [17]:
# Define a new dummy class with a class attribute
class myClass:
    
    myfloat = 6.15
    
    def __init__(self):
        pass

# Create 2 instances of the class
myinst1 = myClass()
myinst2 = myClass()
print('myClass.myfloat =', myClass.myfloat)
print('myinst1.myfloat =', myinst1.myfloat)
print('myinst1.__dict__', myinst1.__dict__)
print('myinst2.myfloat =', myinst2.myfloat)
print('myinst2.__dict__', myinst2.__dict__)
print('')

# Modify the myfloat attribute of the first instance
myinst1.myfloat = 8.18
print('myClass.myfloat =', myClass.myfloat)
print('myinst1.myfloat =', myinst1.myfloat)
print('myinst1.__dict__', myinst1.__dict__)
print('myinst2.myfloat =', myinst2.myfloat)
print('myinst2.__dict__', myinst2.__dict__)

myClass.myfloat = 6.15
myinst1.myfloat = 6.15
myinst1.__dict__ {}
myinst2.myfloat = 6.15
myinst2.__dict__ {}

myClass.myfloat = 6.15
myinst1.myfloat = 8.18
myinst1.__dict__ {'myfloat': 8.18}
myinst2.myfloat = 6.15
myinst2.__dict__ {}


When designing a class it is important to think about which attributes need to be assigned to the class and which to the class instances.

In the following section we will focus on the various types of method attributes.

> Further Reading  
> <a href="https://codesachin.wordpress.com/2016/06/09/the-magic-behind-attribute-access-in-python/" target="_blank">https://codesachin.wordpress.com/2016/06/09/the-magic-behind-attribute-access-in-python/</a>  
> <a href="https://rushter.com/blog/python-class-internals/" target="_blank">https://rushter.com/blog/python-class-internals/</a>

## 2 Methods

---

<br/>

### Functions as Attributes

As briefly mentioned in the previous section, any Python object can be allocated as class attribute. This of course includes functions. To do so we could define a class and function and manually assign the function object to the a class attribute as follows.

In [18]:
# Define a new dummy class
class myClass5:
    pass

# Define a simple function
def say_hello():
    print('Hello!')
    
# Assign the function to the class
myClass5.myfunc = say_hello
# Execute the function
myClass5.myfunc()

# Print the class dictionary
print(myClass5.__dict__)

Hello!
{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'myClass5' objects>, '__weakref__': <attribute '__weakref__' of 'myClass5' objects>, '__doc__': None, 'myfunc': <function say_hello at 0x108af8950>}


We can see that the class is able to call the function and we can also see the object in the class dictionary.

If we only want the function to exist as a class attribute we can simply define the function inside the class. These functions are called *class methods*.

In the following example we define a calculator class with four methods.

In [19]:
# Define a calculator class
class Calculator:
    
    def add(x, y):
        return x + y
    
    def substract(x, y):
        return x - y

    def multiply(x, y):
        return x * y
    
    def divide(x, y):
        return x / y
    
# Print the class dictionary
print(Calculator.__dict__)

{'__module__': '__main__', 'add': <function Calculator.add at 0x108af8c80>, 'substract': <function Calculator.substract at 0x108af8e18>, 'multiply': <function Calculator.multiply at 0x108af8d08>, 'divide': <function Calculator.divide at 0x108af8840>, '__dict__': <attribute '__dict__' of 'Calculator' objects>, '__weakref__': <attribute '__weakref__' of 'Calculator' objects>, '__doc__': None}


These methods work the same as any function, but only exist inside the class.

In [20]:
print('1 + 2 =', Calculator.add(1, 2))
print('5 - 2 =', Calculator.substract(5, 2))
print('7 * 4 =', Calculator.multiply(7, 4))
print('6 / 2 =', Calculator.divide(6, 2))

1 + 2 = 3
5 - 2 = 3
7 * 4 = 28
6 / 2 = 3.0


While this may be useful for grouping certain methods together we don't gain any new functionality. We could have simply defined the functions in separate module.

<br>

### Static Methods

If we create an instance of `Calculator` we cannot use its methods as the instance object is also passed as an argument.

In [21]:
# Create an instance of the class
calc = Calculator()
print(calc.add(1, 2))

TypeError: add() takes 2 positional arguments but 3 were given

To avoid this problem we can declare that the methods are *static*. This can be done using the `@staticmethod` decorator, which essentially tells the method not to expect the instance object as a first argument.

In [22]:
class staticCalculator:
    
    @staticmethod
    def add(x, y):
        return x + y
    
    @staticmethod
    def substract(x, y):
        return x - y
    
    @staticmethod
    def multiply(x, y):
        return x * y
    
    @staticmethod
    def divide(x, y):
        return x / y
    
# Print the class dictionary
print(staticCalculator.__dict__)

{'__module__': '__main__', 'add': <staticmethod object at 0x108bbeb70>, 'substract': <staticmethod object at 0x108bbec50>, 'multiply': <staticmethod object at 0x108bbec88>, 'divide': <staticmethod object at 0x108bbecc0>, '__dict__': <attribute '__dict__' of 'staticCalculator' objects>, '__weakref__': <attribute '__weakref__' of 'staticCalculator' objects>, '__doc__': None}


We can see that the methods are accessable direcly from the class or from an instance.

In [23]:
# Create an instance of the class
calc = staticCalculator()
print('1 + 2 =', staticCalculator.add(1, 2))
print('1 + 2 =', calc.add(1, 2))

1 + 2 = 3
1 + 2 = 3


This is an improvement, but we are still not getting much from the class that we could not already get from individual functions.

### Class Methods

Similarly to static methods, we can use the `@classmethod` decorator to define a class method. This tells the method to expect the class object (`cls`) as the first argument to a given method rather than an instance object. This means class methods have access to all class attributes.

In the following example we define a class for implementing the Stefan-Boltzmann law. 

In [24]:
from math import pi, sqrt

class StefBoltz:
    """ The Stefan–Boltzmann law
    """
    
    # The Stefan-Boltzmann constant
    sigma = 5.670367e-8 # Wm^−2K^−4
        
    @classmethod
    def luminosity(cls, radius, effective_temp):
     
        return 4 * pi * radius ** 2 * cls.sigma * effective_temp ** 4
    
    @classmethod
    def radius(cls, luminosity, effective_temp):
    
        return sqrt(luminosity / (4 * pi * cls.sigma * effective_temp ** 4))
    
    @classmethod
    def temperature(cls, radius, luminosity):
    
        return (luminosity / (4 * pi * radius ** 2 * cls.sigma)) ** 0.25
    
# Print the class dictionary
print(StefBoltz.__dict__)

{'__module__': '__main__', '__doc__': ' The Stefan–Boltzmann law\n    ', 'sigma': 5.670367e-08, 'luminosity': <classmethod object at 0x108bb5400>, 'radius': <classmethod object at 0x108bb5438>, 'temperature': <classmethod object at 0x108aeac50>, '__dict__': <attribute '__dict__' of 'StefBoltz' objects>, '__weakref__': <attribute '__weakref__' of 'StefBoltz' objects>}


We can see that all three methods have access to the class attribute `sigma`, which means that this variable does not have be redefined or passed to each method.

In [25]:
print('The luminosity of the Sun is {:.2e}W'.format(StefBoltz.luminosity(7e8, 5800)))
print('The radius of Alpha Centauri is {:.2e}m'.format(StefBoltz.radius(4e28, 9700)))


The luminosity of the Sun is 3.95e+26W
The radius of Alpha Centauri is 2.52e+09m


Class methods can also be called by class instances.

In [26]:
sb = StefBoltz()
print('The effective temperature of the Earth is {:.2f}K'
      ''.format(sb.temperature(6.37e6, 1.75e+17)))

The effective temperature of the Earth is 278.92K


Finally, class methods can also access static methods and/or other class methods, meaning that a more complicated class structure can be designed that does not require instantitaion.

In [27]:
class Newton:
    
    @staticmethod
    def velocity(displacement, time):
        return displacement / time
    
    @classmethod
    def acceleration(cls, displacement, time):
        return cls.velocity(displacement, time) / time
    
    @classmethod
    def force(cls, mass, displacement, time):
        return mass * cls.acceleration(displacement, time)
    
print('Force = {}N'.format(Newton.force(5, 2, 1)))

Force = 10.0N


While this certainly adds some new functionality, we are still not using class methods to their full potential.

<br>

### Instance Methods

In the previous section you learned how to instantiate a class using the `self` variable. You also saw that (if we don't add a special decorator) when an instance calls a method it expects an extra agrument. You might have already guessed that this argument is `self` and by defining methods with this argument we can pass instance attributes.

This is the default and arguably most powerful way of using methods. In the following example we demonstrate a very simply class for doubling and tripling an input value.


In [30]:
class simpleClass:
    
    def __init__(self, value):
        self.value = value
        
    def double(self):
        return self.value * 2
    
    def triple(self):
        return self.value * 3

# Create a class instance
sc1 = simpleClass(3)
sc2 = simpleClass(7)
print('Instance 1:', sc1.value, sc1.double(), sc1.triple())
print('Instance 2:', sc2.value, sc2.double(), sc2.triple())

Instance 1: 3 6 9
Instance 2: 7 14 21


This may seem trivial, but this allows us to use methods that are automatically tailored to the class instance. The input value only needs to passed once, meaning the code is easier to debug and a single Python object ends up with a lot of functionality.

<br>

### Mixing Methods

In the next example we show that various class methods can be combined to provide more complex behaviour.

In [33]:
class listHandler:
    """ List Handler
    
    Class for handling list properties.
    
    """
    
    def __init__(self, mylist):
        
        self._list = mylist
        self.get_length()
        self.get_first()
        self.get_last()
    
    @staticmethod
    def _get_length(_list):
        
        return len(_list)
    
    @staticmethod
    def _get_element(_list, index):
        
        return _list[index]
    
    @classmethod
    def _get_first(cls, _list):
        
        return cls._get_element(_list, 0)
    
    @classmethod
    def _get_last(cls, _list):
        
        return cls._get_element(_list, -1)
    
    def get_length(self):
        
        self.len = self._get_length(self._list)
        
    def get_first(self):
        
        self.first = self._get_first(self._list)
        
    def get_last(self):
        
        self.last = self._get_last(self._list)

print('listHandler.__dict__', listHandler.__dict__)

listHandler.__dict__ {'__module__': '__main__', '__doc__': ' List Handler\n    \n    Class for handling list properties.\n    \n    ', '__init__': <function listHandler.__init__ at 0x108bc8400>, '_get_length': <staticmethod object at 0x108bc9080>, '_get_element': <staticmethod object at 0x108bc90f0>, '_get_first': <classmethod object at 0x108bc9128>, '_get_last': <classmethod object at 0x108bc9160>, 'get_length': <function listHandler.get_length at 0x108bc8158>, 'get_first': <function listHandler.get_first at 0x108bc80d0>, 'get_last': <function listHandler.get_last at 0x108bc8950>, '__dict__': <attribute '__dict__' of 'listHandler' objects>, '__weakref__': <attribute '__weakref__' of 'listHandler' objects>}


> Note that an underscore is the convention used to denote a "private" attribute. These attributes can be accessed in the usual way, but it is assumed that the user should not need to.

In [32]:
lh = listHandler(list(range(10)))
print('Length:', lh.len)
print('First:', lh.first)
print('Last:', lh.last)

Length: 10
First: 0
Last: 9


This is a very complicated way of doing something simple, but it demonstrates how static, class and instance methods can be used together.

## 3 Properties
---

Now that you are familiar with basic class attributes and methods we can look at class properties.

<br>

### Getters and Setters

As you have already seen, in Python attributes can direcly be assigned to classes or instances. In some cases, however, it may be useful to formally define methods for getting and setting attributes.

In [38]:
class myClass:
    
    def __init__(self):
        self.myattr = None
        
    def get_attr(self):
        return self.myattr
    
    def set_attr(self, value):
        self.myattr = value
        
myinst = myClass()
print('myinst.myattr =', myinst.get_attr())
myinst.set_attr(5)
print('myinst.myattr =', myinst.get_attr())

myinst.myattr = None
myinst.myattr = 5


This may seem silly, as we could have just as easily done the following to get the same result with defining two additional methods.

In [39]:
myinst = myClass()
print('myinst.myattr =', myinst.myattr)
myinst.myattr = 5
print('myinst.myattr =', myinst.myattr)

myinst.myattr = None
myinst.myattr = 5


But now let's assume that we want `myattr` to be a positive integer, but returned as a float and if you attemp to set something else the code will raise an exception.

In [53]:
class myClass:
    
    def __init__(self):
        self.myattr = 1
        
    def get_attr(self):
        return float(self.myattr)
    
    def set_attr(self, value):
        if not isinstance(value, int) or value < 1:
            raise ValueError('myattr must be a positive integer! '
                             'Input value is of type {}.'.format(type(value)))
        self.myattr = value
        
myinst = myClass()
print('myinst.myattr =', myinst.get_attr())
myinst.set_attr(5)
print('myinst.myattr =', myinst.get_attr())

myinst.myattr = 1.0
myinst.myattr = 5.0


We can see that everything works as before, but when we try setting a negative integer 

In [54]:
myinst.set_attr(-4)

ValueError: myattr must be a positive integer! Input value is of type <class 'int'>.

or a float the code raises and exception.

In [55]:
myinst.set_attr(5.6)

ValueError: myattr must be a positive integer! Input value is of type <class 'float'>.

We can see that getters and setters provide a means of adding conditions to the assignment or retrieval of an attribute.

<br>

### Class Properties

In the previous example we saw that we could add conditions to the assignment of an attribute using getters and setters, however there is nothing to prevent a user from directly accessing the attribute and assigning an inccorect value. One way to help avoid this is by using the special `property` decorator.

In [58]:
class myClass:
    
    def __init__(self):
        self._myattr = 1
        
    def get_attr(self):
        return float(self._myattr)
    
    def set_attr(self, value):
        if not isinstance(value, int) or value < 1:
            raise ValueError('myattr must be a positive integer! '
                             'Input value is of type {}.'.format(type(value)))
        self._myattr = value
        
    myattr = property(get_attr, set_attr)
        
myinst = myClass()
print('myinst.myattr =', myinst.myattr)
myinst.myattr = 5
print('myinst.myattr =', myinst.myattr)

myinst.myattr = 1.0
myinst.myattr = 5.0


This allows us to return to the more natural syntax of simply assigning a value to the attribute via the intermediate private attribute `_myattr`.

We can rewrite the same class in the following way for a simple implementation.

In [61]:
class myClass:
    
    def __init__(self):
        self._myattr = 1
    
    @property
    def myattr(self):
        return float(self._myattr)
    
    @myattr.setter
    def myattr(self, value):
        if not isinstance(value, int) or value < 1:
            raise ValueError('myattr must be a positive integer! '
                             'Input value is of type {}.'.format(type(value)))
        self._myattr = value
        
myinst = myClass()
print('myinst.myattr =', myinst.myattr)
myinst.myattr = 5
print('myinst.myattr =', myinst.myattr)

myinst.myattr = 1.0
myinst.myattr = 5.0


Which retains all of the same functionality.

In [62]:
myinst.myattr = 5.6

ValueError: myattr must be a positive integer! Input value is of type <class 'float'>.